In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import shutil
import re

print(tf.__version__)

# Data preperation

In [ ]:
# Create folders
datadirRaw = 'data/Test C'
dirNames = ['0-25', '26-50', '51-75', '76-100']

for dirName in dirNames:
    if not os.path.exists(dirName):
        os.mkdir(datadirRaw + '_' + dirName)
        print("Directory " , dirName ,  " Created ")
    else:    
        print("Directory " , dirName ,  " already exists")

In [ ]:
# Move images into correct folder (based on names)
srcpath = 'data/Video/Test A/'
dataset_size = 4189
quartile_1_directory = "data/Test C_0-25"
quartile_2_directory = "data/Test C_26-50"
quartile_3_directory = "data/Test C_51-75"
quartile_4_directory = "data/Test C_76-100"

fullpath = os.path.join

for dirname, dirnames, filenames in os.walk(srcpath):
    for filename in filenames:
        print(filename)
        source = fullpath(dirname, filename)
        cycle_number = re.search('_(.*).jpg', filename)

        if 0 <= int(cycle_number.group(1)) <= int(dataset_size/4):
            shutil.copy(source, fullpath(quartile_1_directory, filename))

        elif int(dataset_size/4) < int(cycle_number.group(1)) <= int(dataset_size/2):
            shutil.copy(source, fullpath(quartile_2_directory, filename))

        elif int(dataset_size/2) < int(cycle_number.group(1)) <= int(dataset_size/(4/3)):
            shutil.copy(source, fullpath(quartile_3_directory, filename))

        elif int(dataset_size/(4/3)) < int(cycle_number.group(1)) <= dataset_size:
            shutil.copy(source, fullpath(quartile_4_directory, filename))
        
        print('Moved file: ' + filename)

# Model & Training

In [ ]:
# Print a picture
data_dir = 'data' 
data_dir = pathlib.Path(data_dir)
roses = list(data_dir.glob('Test C_0-25/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
# Size of dataset
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)  

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [65]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [66]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [67]:
num_classes = 4

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [68]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [69]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 180, 180, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 90, 90, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 64)       

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()